In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

%matplotlib inline
InteractiveShell.ast_node_interactivity='all'

# Preparing dataset

In [2]:
train_file = pd.read_csv('train.csv')

In [6]:
# split training data into features and labels
x_train = train_file.loc[:, train_file.columns!='label']
y_train = train_file.loc[:, train_file.columns=='label']

In [19]:
# split into training and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=10)

In [20]:
# convert from df to np array
X_train = X_train.to_numpy()
X_valid = X_valid.to_numpy()

In [9]:
# prepare data for dataloader
transform = transforms.Compose([transforms.Resize([32,32])])

class Data(Dataset):
    def __init__(self, x, y):
        self.X = x.reshape(x.shape[0], 1, 28, 28)
        self.X = self.X / 255
        self.X = torch.from_numpy(self.X.astype(np.float32))
        self.X = transform(self.X)
        
        y.resize(y.shape[0],)
        self.Y = torch.from_numpy(y).type(torch.LongTensor)
        
        self.len = self.X.shape[0]
        
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
   
    def __len__(self):
        return self.len

In [21]:
train = Data(X_train, Y_train)
valid = Data(X_valid, Y_valid)

In [22]:
batch_size = 128
trainloader = DataLoader(train, batch_size = batch_size, shuffle = True)
validloader = DataLoader(valid, batch_size = batch_size, shuffle = True)

# Model

In [23]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.fc = nn.Linear(in_features=400, out_features=120)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

# Training

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5()
cost = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [45]:
steps_per_epoch_train = len(trainloader)
epochs = 10

accuracy_list = []
loss_list = []

# iterate through each epoch
for epoch in range(epochs):
    current_loss = 0
    total = 0
    correct = 0
    
    for features, labels in trainloader:
        features = features.to(device)
        labels = labels.to(device)
        
        outputs = model(features)
        loss = cost(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        current_loss += loss.item()
        predicted = outputs.argmax(axis=1).numpy()
        total += labels.size(0)
        correct += (predicted == labels.numpy()).sum().item()
        
    loss_list.append(current_loss/steps_per_epoch)
    accuracy_list.append(100*correct/total)
    
    print(f'epoch: {epoch+1}, loss:{current_loss/steps_per_epoch:.4f}, accuracy: {100*correct/total:.4f}%')

epoch: 1, loss:0.6255, accuracy: 80.9127%
epoch: 2, loss:0.2001, accuracy: 93.9259%
epoch: 3, loss:0.1240, accuracy: 96.1958%
epoch: 4, loss:0.0943, accuracy: 97.1508%
epoch: 5, loss:0.0762, accuracy: 97.6243%
epoch: 6, loss:0.0637, accuracy: 97.9630%
epoch: 7, loss:0.0550, accuracy: 98.2778%
epoch: 8, loss:0.0490, accuracy: 98.4127%
epoch: 9, loss:0.0422, accuracy: 98.6772%
epoch: 10, loss:0.0395, accuracy: 98.7884%


# Predict on Validation Set

In [29]:
pred_list = []
total=0
correct=0

with torch.no_grad():
    for features, labels in validloader:
        outputs = model(features)
        
        predicted = outputs.argmax(axis=1).numpy()
        pred_list.append(predicted)
        total += labels.size(0)
        correct += (predicted == labels.numpy()).sum().item()

print(f'Accuracy: {100 * correct / total:.4f}%')

Accuracy: 98.3095%


# Predict

In [31]:
test_data = pd.read_csv('test.csv')

In [32]:
test_data['fake_labels'] = -1

In [33]:
test = Data(test_data.loc[:, test_data.columns!='fake_labels'].to_numpy(), test_data.fake_labels.to_numpy())
testloader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [34]:
submission = torch.LongTensor()
with torch.no_grad():
    for features, targets in testloader:
        outputs = model(features)
        predictions = outputs.argmax(axis=1)
        submission = torch.cat((submission, predictions), dim=0)

In [35]:
submission_df = pd.DataFrame(submission.numpy(), columns=['Label'])
submission_df.index = [i for i in range(1,len(submission_df)+1)]
submission_df.index.name = 'ImageId'
submission_df.to_csv('submission.csv')